In [1]:
#Celda para librerías


import sklearn as sk

import matplotlib as plt
import matplotlib.pyplot as plt

import pandas as pd
import seaborn as sns
import numpy as np
import plotly.graph_objects as go

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import RobustScaler

import warnings
warnings.filterwarnings('ignore')

print(np.__version__)
from ydata_profiling import ProfileReport
import os
print("Directorio actual:", os.getcwd())



# Have all columns appear when dataframes are displayed.
pd.set_option('display.max_columns', None) 
# Have 100 rows appear when a dataframe is displayed
pd.set_option('display.max_rows', 500)
# Display dimensions whenever a dataframe is printed out.
pd.set_option('display.show_dimensions', True)


#Importando los datos

app_train = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/home-credit-default-risk/application_train.csv')
app_test = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/home-credit-default-risk/application_test.csv')

bureau = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/home-credit-default-risk/bureau.csv')
bureau_balance = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/home-credit-default-risk/bureau_balance.csv')
credit_card_balance = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/home-credit-default-risk/credit_card_balance.csv')
installments_payments = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/home-credit-default-risk/installments_payments.csv')
pos_cash_balance = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/home-credit-default-risk/POS_CASH_balance.csv')
previous_application = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/home-credit-default-risk/previous_application.csv')

app_train.drop(columns=['SK_ID_CURR'], inplace=True)
#Debido a un fallo que detecta que todas los valores de la columna DAYS_BIRTH 
# son nan cuando no es así, vamos a sustituir la columna por una nueva llamada AGE_INT
app_train['AGE_INT'] = app_train['DAYS_BIRTH']/-365
app_train.drop(columns=['DAYS_BIRTH'], inplace=True)


info = app_train.info()
print(info)
describe = app_train.describe()
print(describe)

1.23.5
Directorio actual: c:\Users\Yeray\Desktop\DATA_SCIENCE_ML\Home-Credit-TFG\JUPYTER_NOTEBOOKS
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 121 entries, TARGET to AGE_INT
dtypes: float64(66), int64(39), object(16)
memory usage: 283.9+ MB
None
              TARGET   CNT_CHILDREN  AMT_INCOME_TOTAL    AMT_CREDIT  \
count  307511.000000  307511.000000      3.075110e+05  3.075110e+05   
mean        0.080729       0.417052      1.687979e+05  5.990260e+05   
std         0.272419       0.722121      2.371231e+05  4.024908e+05   
min         0.000000       0.000000      2.565000e+04  4.500000e+04   
25%         0.000000       0.000000      1.125000e+05  2.700000e+05   
50%         0.000000       0.000000      1.471500e+05  5.135310e+05   
75%         0.000000       1.000000      2.025000e+05  8.086500e+05   
max         1.000000      19.000000      1.170000e+08  4.050000e+06   

         AMT_ANNUITY  AMT_GOODS_PRICE  REGION_POPULATION_RELATIVE  \
co

In [2]:
# Modelos base de Machine Learning que vamos a utilizar para hacer pruebas iniciales
#La métrica usada para medir los modelos será el AUC-ROC
# LightGBM
from lightgbm import LGBMClassifier
#KGBoost
from xgboost import XGBClassifier
#KNN
from sklearn.neighbors import KNeighborsClassifier
#Neural Network
from sklearn.neural_network import MLPClassifier
#Random Forest
from sklearn.ensemble import RandomForestClassifier


In [11]:
# Preprocesamiento previo a la creación de los modelos
# Imputación de valores nulos. Vamos a probar de momento a completar variables numéricos con la media
# y las categóricas con la moda.
# Como sabemos que DAYS_EMPLOYEED tiene un valor erroneo, vamos a reemplazarlo por NaN
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

def imputar_valores_nulos(app_train):
    variables_continuas = app_train.select_dtypes(include=['int64', 'float64']).columns 
    for col in variables_continuas:
        app_train[col] = app_train[col].fillna(app_train[col].mean())
    
    variables_categoricas = app_train.select_dtypes(include=['object']).columns
    for col in variables_categoricas:
        app_train[col] = app_train[col].fillna(app_train[col].mode()[0])

    return app_train

app_train_nonan = imputar_valores_nulos(app_train)
#Comprobamos que no haya valores nulos
#print(app_train['DAYS_BIRTH'].unique())
#print(app_train['DAYS_BIRTH'].isnull().sum())
#print(app_train['DAYS_BIRTH'].dtype)

# Verificar si hay valores nulos en app_train después de la imputación
print("Valores nulos en app_train después de la imputación:")
print(app_train_nonan.isnull().sum().sum())

Valores nulos en app_train después de la imputación:
0


In [14]:
# Tratamiento de variables categóricas
# Vamos a probar con Label Encoding para las variables categoricas binarias y con 
# One Hot Encoding para el resto.

variables_categoricas = app_train.select_dtypes(include=['object']).columns
variables_categoricas_binarias = [col for col in variables_categoricas if app_train[col].nunique() == 2]
variables_categoricas_no_binarias = [col for col in variables_categoricas if app_train[col].nunique() > 2]
def label_encoding_binarias(app_train):
    Encoder = LabelEncoder()
    for col in variables_categoricas_binarias:
        app_train[col] = Encoder.fit_transform(app_train[col])
    return app_train

app_train_nonan_le = label_encoding_binarias(app_train_nonan)


def one_hot_encoding_no_binarrias(app_train):
    for col in variables_categoricas_no_binarias:
        app_train = pd.get_dummies(app_train, columns=[col])
    return app_train

app_train_nonan_le_oh = one_hot_encoding_no_binarrias(app_train_nonan_le)
print("Valores nulos en app_train después de la imputación:")
print(app_train_nonan_le_oh.isnull().sum().sum())

Valores nulos en app_train después de la imputación:
0


In [15]:
# Normalización de las variables
# Como gracias a la exploración de datos hemos visto que hay variables que ya han sido normalizadas 
# previamente, nos vamos a centrar en aquellas que necesitan ser normalizadas.

variables_normalizadas = [col for col in app_train.columns if app_train[col].min() >= 0
                          and app_train[col].max() <= 1]    
variables_NO_normalizadas = [col for col in app_train.columns if col not in variables_normalizadas]

# Vamos a probar con Log Transformation por el momento debido a que tienen bastantes valores atípicos
def normalización(app_train):
    for col in variables_NO_normalizadas:
        app_train[col] = np.log1p(app_train[col])
    return app_train

app_train_nonan_le_oh_norm = normalización(app_train_nonan_le_oh)
print("Valores nulos en app_train después de la imputación:")
print(app_train_nonan_le_oh_norm.isnull().sum().sum())

Valores nulos en app_train después de la imputación:
0


In [16]:
#Tras este preprocesamiento inicial vamos a probar diferentes algoritmos. La métrica usada será AUC_ROC.
#Vamos a probar con los siguientes algoritmos:LightGBM, XGBoost, KNN, Neural Network y Random Forest
x = app_train_nonan_le_oh_norm.drop(columns=['TARGET'])
y = app_train_nonan_le_oh_norm['TARGET']

X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [18]:
#LightGBM

# Remove special characters from column names
X_train.columns = X_train.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)
X_test.columns = X_test.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)

lgb_model = LGBMClassifier(random_state=42)
lgb_model.fit(X_train, Y_train)
lgb_predictions = lgb_model.predict_proba(X_test)[:, 1]

#Evaluación
lgb_auc = roc_auc_score(Y_test, lgb_predictions)
print(f"LightGBM AUC: {lgb_auc:.4f}")

[LightGBM] [Info] Number of positive: 19876, number of negative: 226132
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.046946 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10450
[LightGBM] [Info] Number of data points in the train set: 246008, number of used features: 225
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080794 -> initscore=-2.431606
[LightGBM] [Info] Start training from score -2.431606
LightGBM AUC: 0.7536


In [19]:
#XGBoost
xgb_model = XGBClassifier(random_state=42)
xgb_model.fit(X_train, Y_train)
xgb_predictions = xgb_model.predict_proba(X_test)[:, 1]

#Evaluación
xgb_auc = roc_auc_score(Y_test, xgb_predictions)
print(f"XGBoost AUC: {xgb_auc:.4f}")

XGBoost AUC: 0.7491


In [20]:
#KNN 
knn_model = KNeighborsClassifier()
knn_model.fit(X_train, Y_train)
knn_predictions = knn_model.predict_proba(X_test)[:, 1]

#Evaluación
knn_auc = roc_auc_score(Y_test, knn_predictions)
print(f"KNN AUC: {knn_auc:.4f}")

KNN AUC: 0.5482


In [21]:
#Neural Network
nn_model = MLPClassifier(random_state=42)
nn_model.fit(X_train, Y_train)
nn_predictions = nn_model.predict_proba(X_test)[:, 1]

#Evaluación
nn_auc = roc_auc_score(Y_test, nn_predictions)
print(f"Neural Network AUC: {nn_auc:.4f}")

Neural Network AUC: 0.6861


In [22]:
#Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, Y_train)
rf_predictions = rf_model.predict_proba(X_test)[:, 1]

#Evaluación
rf_auc = roc_auc_score(Y_test, rf_predictions)
print(f"Random Forest AUC: {rf_auc:.4f}")

Random Forest AUC: 0.7068


In [23]:
#Extra Trees
from sklearn.ensemble import ExtraTreesClassifier
et_model = ExtraTreesClassifier(random_state=42)
et_model.fit(X_train, Y_train)
et_predictions = et_model.predict_proba(X_test)[:, 1]

#Evaluación
et_auc = roc_auc_score(Y_test, et_predictions) 
print(f"Extra Trees AUC: {et_auc:.4f}")


Extra Trees AUC: 0.6981


In [24]:
#Regresión Logística
lr_model = LogisticRegression(random_state=42)
lr_model.fit(X_train, Y_train)
lr_predictions = lr_model.predict_proba(X_test)[:, 1]

#Evaluación
lr_auc = roc_auc_score(Y_test, lr_predictions)
print(f"Logistic Regression AUC: {lr_auc:.4f}")

Logistic Regression AUC: 0.7417
